# Conv_BoDiEs Evaluation 

## 1. 라이브러리 import

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
import zipfile
import csv
from pathlib import Path


# Google 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# Get CPU or GPU device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
if torch.cuda.is_available():
    print(f'Device name: {torch.cuda.get_device_name(0)}')

Mounted at /content/drive
Using cuda device
Device name: Tesla T4


## 1. Conv_BoDiEs 모델 정의(학습과 똑같이이)

In [2]:
# 모델 정의
class Conv_BoDiEs(nn.Module):
    def __init__(self):
        super(Conv_BoDiEs, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 256, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.flatten = nn.Flatten()

        # Input size is (b, 32, 12, 12) after conv4 and maxpool
        flattened_size = 32 * 12 * 12

        self.fc1 = nn.Sequential(
            nn.Linear(flattened_size, 128),
            nn.ReLU()
        )

        self.fc2 = nn.Linear(128, 16)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor()
])

model = Conv_BoDiEs().to(device)

## 3. 학습과정에서 저장장한 test 데이터로더 경로지정 및 시험결과 저장장

In [4]:
# 저장된 DataLoader 객체 로드
test_data_path = '/content/drive/MyDrive/dataloader/test_data_grayscale.pth'
test_loader = torch.load(test_data_path)

# 모델 로드
model_path = '/content/drive/MyDrive/Conv_BoDiEs/train_results/Best_Conv_BoDiEs_model_1st_grayscale.pth'
model.load_state_dict(torch.load(model_path))
model.eval()  # 평가 모드 설정

# 테스트
criterion = nn.L1Loss()  # 손실 함수

test_loss = 0.0
predictions = []
actuals = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        predictions.extend(outputs.cpu().numpy())
        actuals.extend(labels.cpu().numpy())

test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')

# 파일 경로 설정
result_file_path = '/content/drive/MyDrive/test_results/test_predictions_grayscale_check.csv'

# 디렉토리 생성 (필요시)
Path(result_file_path).parent.mkdir(parents=True, exist_ok=True)

# CSV 파일에 쓰기
with open(result_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Actual', 'Prediction'])
    for actual, prediction in zip(actuals, predictions):
        writer.writerow([actual, prediction])

print(f'Test results saved to {result_file_path}')

<ipython-input-4-d066d4c07f2e>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_loader = torch.load(test_data_path)
<ipython-input-4-d066d4c07f2e>:30: FutureWarning: Y

Test Loss: 0.5537
Test results saved to /content/drive/MyDrive/test_results/test_predictions_grayscale_check.csv
